In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import seaborn as sns
import matplotlib.pyplot as plt

Importación del .csv y eliminación de columnas innecesarias

In [28]:
bcn = pd.read_csv('barcelona_sales.csv')
bcn = bcn.drop(columns=[
'CONSTRUCTIONYEAR',
    'FLOORCLEAN',
    'CADMAXBUILDINGFLOOR',
    'CADDWELLINGCOUNT', 
    'CADASTRALQUALITYID', 
    'BUILTTYPEID_1', 
    'BUILTTYPEID_2', 
    'BUILTTYPEID_3', 
    'LATITUDE', 
    'LONGITUDE', 
    'FLATLOCATIONID', 
    'ISPARKINGSPACEINCLUDEDINPRICE',
    'PARKINGSPACEPRICE',
    'AMENITYID'])

# Se le pone nombre a la última columna. Las 2 últimas columnas son equivalentes a la 'longitud' y 'latitud', pero con los decimales correctos.
bcn = bcn.rename(columns={'geometry':'geo_longitude', 'Unnamed: 42':'geo_latitude'})


Limpieza del DataFrame

In [29]:
# Para eliminar 'c(' de la columna geometry y ')' de la columna geometry_2
# Se definen 2 funciones para limpiar las columnas

# Eliminar 'c(' de la columna geo_longitude:
def remove_first_2(x):
    '''Removes the 2 first characters of the string. Example:
        Input: AAHello
        Output: Hello'''
    return x[2:]

# Eliminar ')' de la columna geo_latitude:
def remove_last_1(x):
    '''Removes the last character of the string. Example:
        Input: Hello)
        Output: Hello'''
    return x[:-1]

In [30]:
bcn['geo_longitude'] = bcn['geo_longitude'].apply(remove_first_2)
bcn['geo_latitude'] = bcn['geo_latitude'].apply(remove_last_1)

In [31]:
# Cálculo del precio por m^2
bcn['UNITPRICE']  = bcn['PRICE'] / bcn['CONSTRUCTEDAREA'] 

# Eliminación de duplicados, basado en el ID
bcn = bcn.drop_duplicates(subset='ASSETID')

Matriz de Correlaciones

In [32]:
bcn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46729 entries, 0 to 61483
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ASSETID                  46729 non-null  object 
 1   PERIOD                   46729 non-null  int64  
 2   PRICE                    46729 non-null  int64  
 3   UNITPRICE                46729 non-null  float64
 4   CONSTRUCTEDAREA          46729 non-null  int64  
 5   ROOMNUMBER               46729 non-null  int64  
 6   BATHNUMBER               46729 non-null  int64  
 7   HASTERRACE               46729 non-null  int64  
 8   HASLIFT                  46729 non-null  int64  
 9   HASAIRCONDITIONING       46729 non-null  int64  
 10  HASPARKINGSPACE          46729 non-null  int64  
 11  HASNORTHORIENTATION      46729 non-null  int64  
 12  HASSOUTHORIENTATION      46729 non-null  int64  
 13  HASEASTORIENTATION       46729 non-null  int64  
 14  HASWESTORIENTATION    

In [26]:
bcn.corr()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12776\1420536391.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  bcn.corr()


,PERIOD,PRICE,UNITPRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,HASTERRACE,HASLIFT,HASAIRCONDITIONING,AMENITYID,...,HASSWIMMINGPOOL,HASDOORMAN,HASGARDEN,ISDUPLEX,ISSTUDIO,ISINTOPFLOOR,CADCONSTRUCTIONYEAR,geo_longitude,geo_latitude,WEIGHTED_PRICE
PERIOD,1.000000,-0.010080,0.020687,-0.020731,-0.013900,-0.010409,-0.057165,-0.025467,0.010741,0.017788,...,-0.008231,-0.016334,-0.021262,-0.002624,-0.003282,-0.010235,-0.046068,0.006815,-0.037650,-0.010080
PRICE,-0.010080,1.000000,0.539064,0.877605,0.493531,0.714820,0.253660,0.260012,0.184036,-0.042210,...,0.182117,0.308178,0.162515,0.079383,-0.077798,0.034028,0.046518,-0.273003,-0.159471,1.000000
UNITPRICE,0.020687,0.539064,1.000000,0.154068,-0.004482,0.297803,0.154259,0.231778,0.236378,-0.038197,...,0.137686,0.184610,0.069458,-0.006868,-0.037540,0.035432,-0.074258,-0.197494,-0.368129,0.539064
CONSTRUCTEDAREA,-0.020731,0.877605,0.154068,1.000000,0.583353,0.699958,0.240507,0.205420,0.116793,-0.034692,...,0.143051,0.260368,0.161261,0.110859,-0.074245,0.025206,0.082326,-0.217254,-0.049265,0.877605
ROOMNUMBER,-0.013900,0.493531,-0.004482,0.583353,1.000000,0.515279,0.119874,0.276266,-0.015282,-0.019863,...,0.039662,0.170825,0.055763,0.011468,-0.336001,-0.023208,0.164957,-0.151574,0.044084,0.493531
BATHNUMBER,-0.010409,0.714820,0.297803,0.699958,0.515279,1.000000,0.217088,0.214455,0.165341,-0.034603,...,0.162103,0.256179,0.142893,0.144938,-0.065613,0.010680,0.114584,-0.212108,-0.082875,0.714820
HASTERRACE,-0.057165,0.253660,0.154259,0.240507,0.119874,0.217088,1.000000,0.113623,0.154959,-0.018212,...,0.111992,0.124417,0.125388,0.111452,-0.032129,0.124229,0.137937,-0.090101,0.004059,0.253660
HASLIFT,-0.025467,0.260012,0.231778,0.205420,0.276266,0.214455,0.113623,1.000000,0.121354,0.000823,...,0.086049,0.163994,0.046115,-0.004846,-0.064933,0.000460,0.328954,-0.092137,-0.016314,0.260012
HASAIRCONDITIONING,0.010741,0.184036,0.236378,0.116793,-0.015282,0.165341,0.154959,0.121354,1.000000,0.007458,...,0.067952,0.100645,0.078113,0.062563,0.011757,0.056141,0.078464,-0.032603,-0.054532,0.184036
AMENITYID,0.017788,-0.042210,-0.038197,-0.034692,-0.019863,-0.034603,-0.018212,0.000823,0.007458,1.000000,...,0.001666,0.053585,0.025056,0.018911,0.016110,0.028859,-0.008614,-0.003052,-0.001368,-0.042210


Weighted Heat Map con los pisos de Barcelona

In [33]:
# Creación columna; precio del inmueble relativo al total
bcn['WEIGHTED_PRICE'] = bcn['PRICE']/bcn['PRICE'].sum()

In [34]:
# Heat Map
bcn['geo_latitude'] = bcn['geo_latitude'].astype(float)
bcn['geo_longitude'] = bcn['geo_longitude'].astype(float)
m = folium.Map(location=[bcn['geo_latitude'].mean(), bcn['geo_longitude'].mean()], zoom_start=10)

HeatMap(data=bcn[['geo_latitude', 'geo_longitude', 'WEIGHTED_PRICE']],
               weight='WEIGHTED_PRICE',
               min_opacity=0.5,
               max_val=15,
               radius=15,
               blur=15,
               max_zoom=1,
               show=False).add_to(m)
m

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12776\1432523328.py:6: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data=bcn[['geo_latitude', 'geo_longitude', 'WEIGHTED_PRICE']],
